In [1]:
import pandas as pd
import numpy as np
import parse_data
import json
import tqdm as tqdm

In [2]:
def load_json_db(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
    return data

def create_dataframe(data):
    df = pd.DataFrame(data=data)
    return df
# Example usage
file_path = "db.sqlite.json"
data = load_json_db(file_path)
df = create_dataframe(data)
df

,id,body,headline,article_name,date_published,date_uploaded
0,1,"The urgency of tackling climate change seems, ...","The left’s belated, and bittersweet, victory o...",the-lefts-belated-and-bittersweet-victory-on-c...,2020-10-15 08:42:11.000000,2022-10-10 06:30:30.489907
1,2,Summer 2022 risks being a ‘perfect storm for g...,Climate change threatens Europe’s once ‘placid...,climate-change-threatens-europes-once-placid-a...,2022-07-04 18:48:55.000000,2022-10-10 06:30:33.769609
2,3,"With Egypt’s COP27 less than six months away, ...",Can MENA countries fight climate change the sa...,can-mena-countries-fight-climate-change-the-sa...,2022-06-11 20:34:08.000000,2022-10-10 06:30:38.083833
3,4,Red Cross says the world has been hit by more ...,Global warming bigger threat than coronavirus:...,climate-change-bigger-threat-than-covid-red-cross,2020-11-17 10:28:01.000000,2022-10-10 06:30:38.588857
4,5,"Therefore, climate action should be central to...",The coronavirus outbreak is part of the climat...,the-coronavirus-outbreak-is-part-of-the-climat...,2020-03-30 12:35:44.000000,2022-10-10 06:30:39.636792
...,...,...,...,...,...,...
8503,8504,Sign in with research by Vanessa Montalbano Go...,Nikki Haley wants to address climate change n...,nikki-haley-wants-address-climate-change-not-b...,2023-02-15 12:38:23.000000,2023-02-16 04:33:42.722584
8504,8505,Sign in No one knows for sure what happened to...,Drought may have doomed this ancient empire — ...,climate-change-drought-history,2023-02-08 16:00:00.000000,2023-02-17 04:24:44.596090
8505,8506,Sign in Both heat and cold can kill. But cold ...,Will global warming make temperature less deadly?,hot-cold-extreme-temperature-deaths,2023-02-16 11:30:48.000000,2023-02-23 03:55:44.057875
8506,8507,Sign in A previous version of this article mis...,How this company plans to use Earth’s heat to ...,geothermal-direct-air-capture-fervo,2023-02-23 12:00:00.000000,2023-02-27 16:34:23.256344


In [10]:
#new_df = pd.DataFrame(columns=['id', 'body', 'headline', 'article_name', 'date_published', 'date_uploaded', 'summary', 'explanation', 'address', 'lat', 'lon'])
new_df = pd.read_csv('main_dataset.csv')
for i in tqdm.tqdm(range(0, len(df))):
    if i % 50 == 0:
        new_df.to_csv(f'main_dataset_{i}.csv')
    article = df.iloc[i]
    body = article['body']

    if len(body) > 12800:
        body = body[:12800]
    
    summary = parse_data.generate_summary(body)
    
    explanation, level = parse_data.extract_explanation(summary)

    locations_df = parse_data.extract_loc(body)

    temp_df = pd.DataFrame(columns=['id', 'body', 'headline', 'article_name', 'date_published', 'date_uploaded', 'summary', 'explanation', 'address', 'lat', 'lon'])
    if len(locations_df) <9:
        for j in range(len(locations_df)):
            temp_df = pd.concat([temp_df, pd.DataFrame({'id': [article['id']], 'body': [article['body']], 'headline': [article['headline']], 'article_name': [article['article_name']], 'date_published': [article['date_published']], 'date_uploaded': [article['date_uploaded']], 'summary': [summary], 'explanation': [explanation], 'address': [locations_df.iloc[j]['address']], 'lat': [locations_df.iloc[j]['latitude']], 'lon': [locations_df.iloc[j]['longitude']]} )], ignore_index=True)
    else:
        for j in range(8):
            temp_df = pd.concat([temp_df, pd.DataFrame({'id': [article['id']], 'body': [article['body']], 'headline': [article['headline']], 'article_name': [article['article_name']], 'date_published': [article['date_published']], 'date_uploaded': [article['date_uploaded']], 'summary': [summary], 'explanation': [explanation], 'address': [locations_df.iloc[j]['address']], 'lat': [locations_df.iloc[j]['latitude']], 'lon': [locations_df.iloc[j]['longitude']]} )], ignore_index=True)
    
    new_df = pd.concat([new_df, temp_df], ignore_index=True)
    

  6%|▌         | 521/8508 [1:28:05<22:30:28, 10.15s/it]


KeyboardInterrupt: 

In [ ]:
df = None
for i in range(50, 550, 50):
    if df is None:
        df = pd.read_csv(f'./main_dataset_{i}.csv')
    else:
        df = pd.concat([df, pd.read_csv(f'./main_dataset_{i}.csv')], ignore_index=True)


In [11]:
df = df.iloc[:, 7:]
df = df.dropna()
rows_to_drop = []
for i in range(len(df)):
    if "Earth" in df.iloc[i, 7]:
        rows_to_drop.append(i)
df.to_csv('big_dataset_2.csv')

,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,id,body,headline,article_name,date_published,date_uploaded,summary,explanation,address,lat,lon
0,0.0,0.0,0.0,0.0,0.0,1,"The urgency of tackling climate change seems, ...","The left’s belated, and bittersweet, victory o...",the-lefts-belated-and-bittersweet-victory-on-c...,2020-10-15 08:42:11.000000,2022-10-10 06:30:30.489907,The urgency of tackling climate change seems t...,1. Emissions: the release of gases into the ai...,United States,39.783730,-100.445882
1,1.0,1.0,1.0,1.0,1.0,1,"The urgency of tackling climate change seems, ...","The left’s belated, and bittersweet, victory o...",the-lefts-belated-and-bittersweet-victory-on-c...,2020-10-15 08:42:11.000000,2022-10-10 06:30:30.489907,The urgency of tackling climate change seems t...,1. Emissions: the release of gases into the ai...,Australia,-24.776109,134.755000
2,2.0,2.0,2.0,2.0,2.0,1,"The urgency of tackling climate change seems, ...","The left’s belated, and bittersweet, victory o...",the-lefts-belated-and-bittersweet-victory-on-c...,2020-10-15 08:42:11.000000,2022-10-10 06:30:30.489907,The urgency of tackling climate change seems t...,1. Emissions: the release of gases into the ai...,中国,35.000074,104.999927
3,3.0,3.0,3.0,3.0,3.0,1,"The urgency of tackling climate change seems, ...","The left’s belated, and bittersweet, victory o...",the-lefts-belated-and-bittersweet-victory-on-c...,2020-10-15 08:42:11.000000,2022-10-10 06:30:30.489907,The urgency of tackling climate change seems t...,1. Emissions: the release of gases into the ai...,Ōu,51.000000,10.000000
4,4.0,4.0,4.0,4.0,4.0,1,"The urgency of tackling climate change seems, ...","The left’s belated, and bittersweet, victory o...",the-lefts-belated-and-bittersweet-victory-on-c...,2020-10-15 08:42:11.000000,2022-10-10 06:30:30.489907,The urgency of tackling climate change seems t...,1. Emissions: the release of gases into the ai...,United Kingdom,54.702354,-3.276575
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
442,NaN,NaN,NaN,NaN,NaN,481,How fundamentally different views about climat...,Donald Trump and California: A battle of wildf...,donald-trump-and-california-a-battle-of-wildfi...,2020-07-30 12:01:38.000000,2022-10-10 07:00:25.099338,The 2020 US election will hinge on differing v...,1. Emissions Regulations: Rules and laws that ...,"Hollywood, Los Angeles, Los Angeles County, CA...",34.098003,-118.329523
443,NaN,NaN,NaN,NaN,NaN,487,A much less costly way to regenerate our fores...,Reforestation is not necessarily about plantin...,reforestation-is-not-necessarily-about-plantin...,2020-01-18 11:00:56.000000,2022-10-10 07:00:54.286876,"Assisted natural regeneration, a low-tech and ...",1. Assisted natural regeneration - This means ...,Brasil,-10.333333,-53.200000
444,NaN,NaN,NaN,NaN,NaN,492,"Publication by the UN, Red Cross says 38 heatw...",Future heatwaves will lead to large ‘loss of l...,future-heatwaves-will-lead-to-large-loss-of-li...,2022-10-10 13:46:17.000000,2022-10-17 21:12:18.867964,A report by the United Nations and the Red Cro...,1. Unsustainable - This means that something c...,Asia,51.208697,89.234375
445,NaN,NaN,NaN,NaN,NaN,516,The nomads who have travelled around Mongolia’...,"In Mongolia, climate crisis threatens herding ...",in-mongolias-tsaikhir-valley-climate-risks-anc...,2022-11-29 01:59:12.000000,2022-12-03 11:43:34.468105,Nomadic herders in Mongolia's Tsaikhir Valley ...,1. Drought - a prolonged period of dry weather...,Монгол улс ᠮᠤᠩᠭᠤᠯ ᠤᠯᠤᠰ,46.825039,103.849974


In [10]:

def df_to_sqlite_json(df, file_path):
    # Create a dictionary to store the table data
    table_data = []

    # Iterate over each row in the DataFrame
    for index, row in df.iterrows():
        # Convert the row data to a dictionary
        row_data = row.to_dict()
        # Add the row data to the table data list
        table_data.append(row_data)

    # Create a dictionary to store the table schema
    table_schema = {}
    for column in df.columns:
        table_schema[column] = str(df[column].dtype)

    # Create a dictionary to store the table metadata
    table_metadata = {
        "name": df.name,
        "schema": table_schema
    }

    # Create a dictionary to store the database metadata
    db_metadata = {
        "tables": [table_metadata]
    }

    # Write the database metadata and table data to the JSON file
    with open(file_path, 'w') as f:
        json.dump(table_data, f)
        f.write('\n')
        json.dump(db_metadata, f)

# Example usage
df = pd.DataFrame({
    "name": ["John", "Jane", "Bob"],
    "age": [25, 30, 40],
    "location": ["New York", "Los Angeles", "Chicago"]
})

df.name = "example_table"
df_to_sqlite_json(df, "example.sqlite.json")

TypeError: Object of type Series is not JSON serializable